<center><h1>TMDB Movies</h1></center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TMDB-Movies" data-toc-modified-id="TMDB-Movies-1">TMDB Movies</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-2">Introduction</a></span></li><li><span><a href="#Data-Wrangling" data-toc-modified-id="Data-Wrangling-3">Data Wrangling</a></span><ul class="toc-item"><li><span><a href="#Gather" data-toc-modified-id="Gather-3.1">Gather</a></span></li><li><span><a href="#Assess" data-toc-modified-id="Assess-3.2">Assess</a></span></li><li><span><a href="#Clean" data-toc-modified-id="Clean-3.3">Clean</a></span></li><li><span><a href="#Saving-to-local-file" data-toc-modified-id="Saving-to-local-file-3.4">Saving to local file</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-4">Exploratory Data Analysis</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5">Conclusion</a></span></li><li><span><a href="#References" data-toc-modified-id="References-6">References</a></span></li></ul></div>

# Introduction

This project started as an assessment for the **Udacity Data Analysis Advanced Track**.

The Movie Database (TMDb) is a community built movie and TV database dating back to 2008.
This data set contains information about 10,000 movies collected from The Movie Database (TMDb),including user ratings and revenue.


# Data Wrangling

We begin by importing the libraries that we will be using for our analysis:
- [Numpy](https://numpy.org/)
- [Pandas](https://pandas.pydata.org/)
- [Matplotlib](https://matplotlib.org/stable/index.html#)
- [Seaborn](https://seaborn.pydata.org/)
- [Requests](https://2.python-requests.org/en/master/)
- [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)

In [1]:
# importing required libraries
from bs4 import BeautifulSoup
import requests
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Gather

### TMDB Data

We can read the TMDB data directly from the `tmdb-movies.csv` file

The final two columns ending with “_adj” show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time.


In [2]:
df_tmdb = pd.read_csv('tmdb-movies.csv', index_col='id')
df_tmdb.sample(3)

,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,keywords,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
id,,,,,,,,,,,,,,,,,,,,
8850,tt0111143,0.843170,25000000,48063435,The Shadow,Alec Baldwin|John Lone|Penelope Ann Miller|Pet...,NaN,Russell Mulcahy,The Shadow Knows!,new york|atomic bomb|ladykiller|hypnosis|super...,"Based on the 1930's comic strip, puts the hero...",108,Adventure|Fantasy|Action|Thriller|Science Fiction,Universal Pictures,7/1/94,81,5.3,1994,3.677779e+07,7.070667e+07
10299,tt0058213,0.015476,0,0,"Hush... Hush, Sweet Charlotte",Bette Davis|Olivia de Havilland|Agnes Moorehea...,NaN,Robert Aldrich,The years will soon erase a lover's lies...the...,southern u.s.|mental disease|highway|murder|re...,Aged and wealthy Charlotte Hollis has lived al...,133,Crime|Drama|Horror|Mystery|Thriller,Twentieth Century Fox Film Corporation|The Ass...,12/15/64,14,5.3,1964,0.000000e+00,0.000000e+00
117263,tt2302755,2.063266,70000000,161025640,Olympus Has Fallen,Gerard Butler|Aaron Eckhart|Angela Bassett|Mor...,NaN,Antoine Fuqua,When our flag falls our nation will rise.,white house|secret service|terrorist attack,"When the White House (Secret Service Code: ""Ol...",120,Action|Thriller,Nu Image Films|Millennium Films,3/20/13,2403,6.2,2013,6.552233e+07,1.507254e+08


In [3]:
df_tmdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10866 entries, 135397 to 22293
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               10856 non-null  object 
 1   popularity            10866 non-null  float64
 2   budget                10866 non-null  int64  
 3   revenue               10866 non-null  int64  
 4   original_title        10866 non-null  object 
 5   cast                  10790 non-null  object 
 6   homepage              2936 non-null   object 
 7   director              10822 non-null  object 
 8   tagline               8042 non-null   object 
 9   keywords              9373 non-null   object 
 10  overview              10862 non-null  object 
 11  runtime               10866 non-null  int64  
 12  genres                10843 non-null  object 
 13  production_companies  9836 non-null   object 
 14  release_date          10866 non-null  object 
 15  vote_count    

### IMDB Data

To compare the TMDB ratings with the IMDB ratings. we can scrape the IMDB ratings using their `imdb_id` provided from the TMDB database.

It was necessary to save the scraped data to a file on disk `imdb-movies.csv` as the scraping process took around 3 hours and 49 minuntes.

In [ ]:
%time
imdb_ids = df_tmdb[df_tmdb['imdb_id'].notnull()]['imdb_id']
imdb_file = 'imdb-movies.csv'

df_imdb = pd.DataFrame()


if os.path.exists(imdb_file):
    print(f'File {imdb_file} already exists')
    df_imdb = pd.read_csv(imdb_file, index_col=0)
else:
    fails = {}

    for id,imdb_id in imdb_ids.items():
        try:
            print(f'{id} : {imdb_id}')
            url = "https://www.imdb.com/title/" + str(imdb_id) + "/"
            print(url)
            r = requests.get(url=url)
            # create a BeautifulSoup object
            soup = BeautifulSoup(r.text, 'html.parser')
            imdb_rating = float(
                soup.find('span', {"itemprop": "ratingValue"}).text)
            data = {
                'id': id,
                'imdb_id': imdb_id,
                'imdb_rating': imdb_rating
            }
            df_imdb = df_imdb.append(data, ignore_index=True)
            print(f'{id} : {imdb_rating}')
        except Exception:
            print(f'{id} : Failed to fetch.')
            fails[id] = Exception
    print(fails)
    df_imdb['id'] = df_imdb['id'].astype(int)
    df_imdb.to_csv(imdb_file, index=False)

df_imdb.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
135397 : tt0369610
https://www.imdb.com/title/tt0369610/
135397 : 7.0
76341 : tt1392190
https://www.imdb.com/title/tt1392190/
76341 : 8.1
262500 : tt2908446
https://www.imdb.com/title/tt2908446/
262500 : 6.2
140607 : tt2488496
https://www.imdb.com/title/tt2488496/
140607 : 7.9
168259 : tt2820852
https://www.imdb.com/title/tt2820852/
168259 : 7.1
281957 : tt1663202
https://www.imdb.com/title/tt1663202/
281957 : 8.0
87101 : tt1340138
https://www.imdb.com/title/tt1340138/
87101 : 6.3
286217 : tt3659388
https://www.imdb.com/title/tt3659388/
286217 : 8.0
211672 : tt2293640
https://www.imdb.com/title/tt2293640/
211672 : 6.4
150540 : tt2096673
https://www.imdb.com/title/tt2096673/
150540 : 8.1
206647 : tt2379713
https://www.imdb.com/title/tt2379713/
206647 : 6.8
76757 : tt1617661
https://www.imdb.com/title/tt1617661/
76757 : 5.3
264660 : tt0470752
https://www.imdb.com/title/tt0470752/
264660 : 7.7
257344 : tt2120120
https://www.i

## Assess

In [ ]:
df_tmdb[df_tmdb.duplicated()]

In [ ]:
df_tmdb[df_tmdb['original_title'] == 'TEKKEN']

Only one row is duplicated which is the title 'TEKKEN' with the exact same data.So, it can be safely removed.

In [ ]:
df_tmdb.isnull().any()

The columns 'imdb_id', 'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview', 'genres' and 'production_companies' have NaN values.

In [ ]:
df_tmdb.describe()

In [ ]:
# check for entries where budget or revenue equals 0
df_tmdb.query(
    'budget == 0 or revenue == 0 or budget_adj == 0 or revenue_adj == 0 or runtime == 0').info()

7011 rows have 0 budget or revenue or 0 runtime.

In [ ]:
df_tmdb.dtypes

### Quality

1. missing data in the following columns: `imdb_id`, `cast`, `homepage`, `director`, `tagline`, `keywords`, `overview`,  `genres` and `production_companies`
2. Zero values found in columns: `budget`, `revenue`, `budget_adj`, `revenue_adj` and `runtime`.
3. The columns `cast`, `keywords` `genres` and `production_companies` contains strings separated by '|' instead of a list of items.
4. `release_date` column is of type string instead of datetime. 

### Tidiness
1. The columns `cast`, `keywords` `genres` and `production_companies` values exist as one entry. Needs adjustments to be able to analyze each cast member, keyword, genre or production company separately.
2. The IMDB DataFrame should be merged with the original TMDB DataFrame so that every row represent an observation.

## Clean

#### Define

- Remove Duplicates using pandas drop_duplicates function.

#### Code

In [ ]:
df_clean = df_tmdb.drop_duplicates()

#### Test

In [ ]:
df_clean.duplicated().any()

#### Define

Drop the columns that are not relevant to our analysis which are: `homepage`,`director`, `tagline`, `overview`.

#### Code

In [ ]:
df_clean = df_tmdb.drop(
    ['homepage', 'director', 'tagline', 'overview'], axis=1)

#### Test

In [ ]:
df_clean.columns

#### Define

Splitting the  `cast`, `keywords`, `genres` and `production_companies` columns using the str.split function with '|' as the separator.

Then we collect the unique values in all the columns by defining a custom function `to_1D` that converts the multi-dimensional arrays into one dimension and then getting the value counts of each column.

#### Code

In [ ]:
# the columns to split
cols = ['cast', 'keywords', 'genres', 'production_companies']
# a dictionary to store unique values of each column
unique = {}


def to_1D(series):
    """
    converts a 2D series to a 1D series
    """
    return pd.Series([x for _list in series for x in _list], dtype='object')


for col in cols:
    df_clean[col] = df_clean[col].str.split('|')
    unique[col] = to_1D(df_clean[col].dropna()).value_counts().index

#### Test

In [ ]:
df_clean.sample(3)

In [ ]:
unique

#### Define

- fix the runtime zero values by replacing them with the mean value.

#### Code

In [ ]:
df_clean['runtime'].value_counts().sort_index()

In [ ]:
df_clean['runtime'].replace(0, df_clean['runtime'].mean(), inplace=True)

#### Test

In [ ]:
df_clean[df_clean['runtime'] == 0]

#### Define

- we can use the same technique for the budget and revenue columns

#### Code

In [ ]:
for col in ['budget', 'revenue', 'budget_adj', 'revenue_adj']:
    df_clean[col].replace(0, df_clean[col].mean(), inplace=True)

#### Test

In [ ]:
for col in ['budget', 'revenue', 'budget_adj', 'revenue_adj']:
    print(df_clean[df_clean[col] == 0][col].count())

#### Define

- Extract the release month from the release date by splitting into columns using the str.split function with '/' as the separator and get only the month as the date format is MM/DD/YY.

#### Code

In [ ]:
df_clean['release_month'] = df_clean['release_date'].str.split(
    '/', expand=True)[0]


In [ ]:
# convert the release month column to Integer type
df_clean['release_month'] = df_clean['release_month'].astype(int)

#### Test

In [ ]:
df_clean['release_month']

#### Define



#### Code

In [ ]:
df_clean = df_clean.merge(df_imdb,on='imdb_id')

#### Test

In [ ]:
df_clean.info()

In [ ]:
df_clean['imdb_rating'].describe()

#### Define



#### Code

In [ ]:
df_clean.info()

#### Test

## Saving to local file

Now that we have successfully cleaned our data, we need to save to and external file to use it later in our analysis.

In [ ]:
df_clean.to_csv('tmdb-clean.csv', index=True)

# Exploratory Data Analysis

In [ ]:
df = pd.read_csv('tmdb-clean.csv', index_col='id')

In [ ]:
df.head()

In [ ]:
df.hist(figsize=(16, 12));

In [ ]:
df['genres'].value_counts()

In [ ]:
df.groupby(['genres'])['vote_average'].mean()

In [ ]:
# gat a list of all the genres
genres = []
for item in df['genres'].dropna().values:
    all = item.split('|')
    for genre in all:
        if genre not in genres:
            genres.append(genre)

genres

In [ ]:
# gat a list of all the gproduction companies
companies = []
for item in df['production_companies'].dropna().values:
    all = item.split('|')
    for company in all:
        if company not in companies:
            companies.append(company)

len(companies)

In [ ]:
df.groupby(['release_year'])['popularity'].mean().plot(figsize=(16, 10))

In [ ]:
df.groupby(['release_year'])['vote_average'].mean().plot(figsize=(16, 10))

In [ ]:
df.groupby(['release_year'])[['budget', 'revenue']
                             ].mean().plot(figsize=(16, 10))

In [ ]:
sns.boxplot(x=df['vote_average'])

In [ ]:
# calculate release month
df['release_month'] = df['release_date'].str.split(
    '/', expand=True)[0].astype(int)
df['release_month'].value_counts().sort_index().plot(kind='bar', figsize=(16, 6))

# Conclusion

Which genres are
most popular from
year to year? What
kinds of properties
are associated with
movies that have high
revenues?


# References

- [Scrape IMDB movie rating and details using python](https://dev.to/magesh236/scrape-imdb-movie-rating-and-details-3a7c)
- [pandas.Series.str.split](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html#pandas.Series.str.split)
- [Dealing with List Values in Pandas Dataframes](https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173)
- [Are you confused using Series.str.contains() and DataFrame.isin() then look into this article](https://medium.com/analytics-vidhya/filter-pandas-dataframe-rows-by-a-list-of-strings-e95c225822fa)